In [ ]:
from google.colab import files
files.upload()
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c intro-dl-2024
!unzip /content/intro-dl-2024.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/train/03145.jpg   
  inflating: train/train/03146.jpg   
  inflating: train/train/03147.jpg   
  inflating: train/train/03148.jpg   
  inflating: train/train/03149.jpg   
  inflating: train/train/03150.jpg   
  inflating: train/train/03151.jpg   
  inflating: train/train/03152.jpg   
  inflating: train/train/03153.jpg   
  inflating: train/train/03154.jpg   
  inflating: train/train/03155.jpg   
  inflating: train/train/03156.jpg   
  inflating: train/train/03157.jpg   
  inflating: train/train/03158.jpg   
  inflating: train/train/03159.jpg   
  inflating: train/train/03160.jpg   
  inflating: train/train/03161.jpg   
  inflating: train/train/03162.jpg   
  inflating: train/train/03163.jpg   
  inflating: train/train/03164.jpg   
  inflating: train/train/03165.jpg   
  inflating: train/train/03166.jpg   
  inflating: train/train/03167.jpg   
  inflating: train/train/03168.jpg   
  inflating: train/t

In [ ]:
!kaggle datasets download  matewl/crops-cars196


Dataset URL: https://www.kaggle.com/datasets/matewl/crops-cars196
License(s): unknown
100% 1.98G/1.98G [00:33<00:00, 65.6MB/s]
100% 1.98G/1.98G [00:33<00:00, 64.4MB/s]


In [ ]:
!unzip /content/crops-cars196.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train_crops/train_crops/03145.jpg  
  inflating: train_crops/train_crops/03146.jpg  
  inflating: train_crops/train_crops/03147.jpg  
  inflating: train_crops/train_crops/03148.jpg  
  inflating: train_crops/train_crops/03149.jpg  
  inflating: train_crops/train_crops/03150.jpg  
  inflating: train_crops/train_crops/03151.jpg  
  inflating: train_crops/train_crops/03152.jpg  
  inflating: train_crops/train_crops/03153.jpg  
  inflating: train_crops/train_crops/03154.jpg  
  inflating: train_crops/train_crops/03155.jpg  
  inflating: train_crops/train_crops/03156.jpg  
  inflating: train_crops/train_crops/03157.jpg  
  inflating: train_crops/train_crops/03158.jpg  
  inflating: train_crops/train_crops/03159.jpg  
  inflating: train_crops/train_crops/03160.jpg  
  inflating: train_crops/train_crops/03161.jpg  
  inflating: train_crops/train_crops/03162.jpg  
  inflating: train_crops/train_crops/03163.jpg  
 

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import os
import pandas as pd
import torch
import cv2
from torch.utils.data import Dataset
import timm
import sys
from datetime import datetime, timedelta
import torch.nn as nn
from sklearn.model_selection import train_test_split

import albumentations as A
import torch.nn.functional as F

IMG_SIZE = 500

torch.cuda.is_available()

True

In [ ]:
class CarDataset(Dataset):
    def __init__(self, df_filename, phase, dataroot='./train/', transforms=None):
        if isinstance(df_filename, str):
            self.df = pd.read_csv(df_filename)
        else:
            self.df = df_filename
        self.phase = phase
        self.dataroot = dataroot
        self.transforms = transforms

    def __getitem__(self, idx):
        filename = self.df.iloc[idx]['filename']
        label = self.df.iloc[idx]['label']
        image = cv2.imread((os.path.join(self.dataroot, self.phase, filename)))

        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

        if self.transforms is not None:
            augmented = self.transforms(image=image)
            image = augmented['image']

        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE)) # тут можно увеличить

        image = self._preprocess(image)

        return image, label

    def __len__(self):
        return len(self.df)

    def _preprocess(self, image):
        image = image.transpose(2, 0, 1)
        image = (image - 127.5) / 127.5
        image = torch.from_numpy(image).float()

        return image


In [ ]:
class LabelSmoothingCrossEntropy(nn.Module):
    """ NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        assert smoothing < 1.0
        self.smoothing = smoothing
        self.confidence = 1. - smoothing

    def forward(self, x: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()



In [ ]:
train_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Blur(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.CLAHE(p=0.5),
    A.ChannelShuffle(p=0.5),
    A.ColorJitter(p=0.5),
    A.ChannelDropout(p=0.5),
    A.RandomGamma(p=0.5),
    A.RGBShift(p=0.5),
    A.Downscale(scale_min=0.9, scale_max=0.99, p=0.5),
    A.GaussNoise(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.ToSepia(p=0.5),
    A.Emboss(p=0.5),
    A.Equalize(p=0.5),
    A.MultiplicativeNoise(p=0.5),
    A.ToGray(p=0.5)
])


/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1692: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  warnings.warn(


In [ ]:
def print_one_line(s):
    time_string = datetime.now().strftime('%H:%M:%S')
    sys.stdout.write('\r' + time_string + ' ' + s)
    sys.stdout.flush()


In [ ]:
df = pd.read_csv('./train.csv')
train_df, test_df = train_test_split(df, test_size=0.2, random_state=1337, stratify=df['label'])

train_dataset = CarDataset(df_filename=train_df,
                           phase='train',
                           transforms=train_aug)

val_dataset = CarDataset(df_filename=test_df,
                         phase='train')

train_dataset_crop = CarDataset(df_filename=train_df,
                                phase='train_crops',
                                dataroot='./train_crops',
                                transforms=train_aug)

val_dataset_crop = CarDataset(df_filename=test_df,
                                dataroot='./train_crops',
                              phase='train_crops')

full_dataset_crop = CarDataset(df_filename=df,
                                dataroot='./train_crops',
                              phase='train_crops')


In [ ]:
train_dataset_crop.__len__()

6515

In [ ]:
val_dataset_crop.__len__()

1629

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=13, shuffle=True,
    num_workers=0, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=13, shuffle=False,
    num_workers=0, pin_memory=True)

train_loader_crop = torch.utils.data.DataLoader(
    train_dataset_crop, batch_size=1, shuffle=True,
    num_workers=0, pin_memory=True)

val_loader_crop = torch.utils.data.DataLoader(
    val_dataset_crop,
    batch_size=1
    , shuffle=False,
    num_workers=0, pin_memory=True)


full_loader_crop = torch.utils.data.DataLoader(
    full_dataset_crop,
    batch_size=32, shuffle=True,
    num_workers=0, pin_memory=True)

NameError: name 'torch' is not defined

In [ ]:
for i in

In [ ]:
# res_next_50 = timm.create_model('timm/resnext50_32x4d.a3_in1k', pretrained=True, drop_rate=0.5)
# res_next_50.__name__ = 'res_next_50'

# res_next_101 = timm.create_model('timm/resnext101_64x4d.c1_in1k', pretrained=True, drop_rate=0.5)
# res_next_101.__name__ = 'res_next_101'

# resnet_50 = timm.create_model('resnet50', pretrained=True, drop_rate=0.5)
# resnet_50.__name__ = 'resnet_50'

# resnet_101 = timm.create_model('resnet101', pretrained=True, drop_rate=0.5)
# resnet_101.__name__ = 'resnet_101'

resnet_150 = timm.create_model('resnet152.a3_in1k', pretrained=True, drop_rate=0.5)
resnet_150.__name__ = 'resnet_150'
resnet_150.reset_classifier(196)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

In [ ]:
for i in

In [ ]:
def train(model, load_weights=False, lr = 3e-4, is_crop=False, best_val_acc=0):

    crop = ''
    dp = str(int(model.drop_rate * 10))
    model_path = 'best_val_resnet_150_224_dp_0_.pth'
    model.reset_classifier(196)
    if load_weights:
        try:
            model.load_state_dict(torch.load(model_path, torch.device('cpu')))
            print('Weights have been loaded!')
        except:
            print('There are no saved weights!')

    optimizer = torch.optim.Adam(model.parameters(), lr)
    ce_loss = LabelSmoothingCrossEntropy()

    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, min_lr=1e-6,
                                                          verbose=True)





    if is_crop:
        t_loader = train_loader_crop
        v_loader = val_loader_crop
    else:
        t_loader = train_loader
        v_loader = val_loader

    model = model.cuda()

    k = 0
    for epoch in range(1, 1000):
        start_time = datetime.now()

        train_accuracy = 0
        train_loss = 0
        total = 0
        correct = 0
        batch_idx = 0

        model.train()
        for inputs, targets in t_loader:
            k += 1
            inputs, targets = inputs.cuda(), targets.cuda()

            outputs = model(inputs)

            loss = ce_loss(outputs, targets)
            train_loss += loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

            batch_idx += 1

            train_accuracy = float(100.0 * correct) / total

            print_one_line('Epoch {} Loss {:.4f} | Acc={:.2f}% ({}/{})'.format(epoch,
                                                                            train_loss / batch_idx,
                                                                            train_accuracy,
                                                                            correct, total))

        print('')
        model.eval()
        total = 0
        correct = 0
        valid_loss = 0
        val_batch = 0
        with torch.no_grad():
            for img, targets in v_loader:
                targets = targets.cuda()
                inputs = img.cuda()

                outputs = model(inputs)
                val_loss = ce_loss(outputs, targets)
                valid_loss += val_loss
                val_batch += 1

                _, predicted = torch.max(outputs, 1)

                total += targets.size(0)
                correct += predicted.eq(targets.data).cpu().sum()

            val_accuracy = float(100.0 * correct) / total
            valid_loss = valid_loss / val_batch

            lr_scheduler.step(val_accuracy)


            print('Val accuracy {:.4f}'.format(val_accuracy))
            print('Validation loss {:.4f}:'.format(valid_loss))

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            torch.save(model.state_dict(), model_path)


In [ ]:
train(resnet_150, load_weights=False, is_crop=False)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


18:02:03 Epoch 1 Loss 5.2817 | Acc=0.63% (41/6515)
Val accuracy 1.0436
Validation loss 5.2384:
18:21:46 Epoch 2 Loss 4.8316 | Acc=4.70% (306/6515)
Val accuracy 20.9331
Validation loss 3.8586:
18:41:28 Epoch 3 Loss 3.4267 | Acc=28.70% (1870/6515)
Val accuracy 57.2130
Validation loss 2.5068:
19:01:23 Epoch 4 Loss 2.5318 | Acc=54.23% (3533/6515)
Val accuracy 72.7440
Validation loss 1.9845:
19:20:54 Epoch 5 Loss 2.1053 | Acc=67.75% (4414/6515)
Val accuracy 77.3481
Validation loss 1.7989:
19:40:34 Epoch 6 Loss 1.8624 | Acc=76.16% (4962/6515)
Val accuracy 81.5838
Validation loss 1.6422:
20:00:04 Epoch 7 Loss 1.6892 | Acc=81.43% (5305/6515)


KeyboardInterrupt: 

In [ ]:
def train_full(model, load_weights=False, lr = 3e-4, is_crop=False):

    crop = '_crop_'
    dp = str(int(model.drop_rate * 10))
    model_path = '3best_val_resnet_150_224_dp_0_crop_.pth'
    model.reset_classifier(196)
    if load_weights:
        try:
            model.load_state_dict(torch.load(model_path, torch.device('cpu')))
            print('Weights have been loaded!')
        except:
            print('There are no saved weights!')

    optimizer = torch.optim.Adam(model.parameters(), lr)
    ce_loss = LabelSmoothingCrossEntropy()

    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, min_lr=1e-6,
                                                          verbose=True)





    if is_crop:
        t_loader = full_loader_crop
    else:
        t_loader = full_loader

    model = model.cuda()

    k = 0
    for epoch in range(1, 1000):
        start_time = datetime.now()

        train_accuracy = 0
        train_loss = 0
        total = 0
        correct = 0
        batch_idx = 0

        model.train()
        for inputs, targets in t_loader:
            k += 1
            inputs, targets = inputs.cuda(), targets.cuda()

            outputs = model(inputs)

            loss = ce_loss(outputs, targets)
            train_loss += loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

            batch_idx += 1

            train_accuracy = float(100.0 * correct) / total

            print_one_line('Epoch {} Loss {:.4f} | Acc={:.2f}% ({}/{})'.format(epoch,
                                                                            train_loss / batch_idx,
                                                                            train_accuracy,
                                                                            correct, total))

        torch.save(model.state_dict(), str(4 + epoch) + model_path)


In [ ]:

train_full(resnet_150, load_weights=True, is_crop=True)

In [ ]:
model_path = 'best_val_resnet_150_224_dp_0_crop_.pth'

torch.save(resnet_150.state_dict(), '3' + model_path)

In [ ]:
model_path = 'best_val_resnet_150_224_dp_0_crop_.pth'

torch.save(resnet_150.state_dict(), str(2) + model_path)


In [ ]:
from torch.utils.data import DataLoader, random_split


In [ ]:
from torch.utils.data import DataLoader, random_split

df = pd.read_csv('sample_submission.csv')

train_dataset = CarDataset(df_filename=df,
                           phase='test')

train_dataset_crop = CarDataset(df_filename=df,
                                phase='test_crops',
                                dataroot='./test_crops')


test_loader = DataLoader(dataset=train_dataset_crop, batch_size=512, shuffle=False, num_workers=0)

model = resnet_150
model.reset_classifier(196)

model_path = '93best_val_resnet_150_224_dp_0_crop_.pth'
model.load_state_dict(torch.load(model_path, torch.device('cpu')))

model.eval()
model.cuda()

predictions = []
with torch.no_grad():
    for features, _, in test_loader:
        outputs = model(features.cuda())
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

# print(predictions)
df = pd.read_csv('sample_submission.csv')
df['label'] = predictions
df.to_csv(model_path + '.csv', index=False)
